In [30]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

In [31]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [32]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [33]:
model_df = df.copy()

In [34]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [35]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino,"[we, just, wanna, party, party, just, for, you..."
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino,"[this, is, america, don't, catch, you, slippin..."
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino,"[this, is, america, (skrrt, skrrt, woo), don't..."
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino,"[ooh-ooh-ooh-ooh-ooh, tell, somebody, you, go,..."
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino,"[this, is, america, (woo, ayy), don't, catch, ..."


In [36]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [37]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [38]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

0    we just wanna party party just for you we just...
1    this is america dont catch you slippin now don...
2    this is america dont catch you slippin now loo...
3    tell somebody you go tell somebody grandma tol...
4    this is america dont catch you slippin now don...
Name: verse, dtype: object

In [39]:
verses = list(model_df.verse.values)

In [40]:
verses[:10]

['we just wanna party party just for you we just want the money money just for you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame we just wanna party party just for you we just want the money money just for you you i know you wanna party party just for free girl you got me dancin girl you got me dancin dance and shake the frame',
 'this is america dont catch you slippin now dont catch you slippin now look what im whippin now this is america dont catch you slippin now dont catch you slippin now look what im whippin now',
 'this is america dont catch you slippin now look how im livin now police be trippin now this is america guns in my area word my area i got the strap i gotta carry ima go into this ugh this is guerilla ima go get the bag or ima get the pad im so cold like im so dope like we gon blow like straight up',
 'tell somebody you go tell somebody grandma told me get your money black man get your black man get yo

In [41]:
all_verses_all_rappers = ','.join(verses)

all_verses_all_rappers[:100]

'we just wanna party party just for you we just want the money money just for you i know you wanna pa'

In [42]:
file = all_verses_all_rappers

In [43]:
# Read in text and change unicode characters to ASCII
import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

file_len = len(file)
print(f'file_len = {file_len}')

file_len = 2670560


In [44]:
chunk_len = 400

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

lamor who dress like sarah jessica and live like princess diana so often they have addictions and im the one thatll feed it but truly youre bad enough that you dont even really need it you can show up at the party on dirty public transit and i guarantee the cameraman will still be snappin candids of you and your posse partying drinking watch you get handed with your virgin island tans all lookin li


In [45]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, hidden):
        output = self.encoder(input.view(1, -1))
        output, hidden = self.gru(output.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        
        return output, hidden
    
    def init_hidden(self):
        return Variable(torch.randn(self.n_layers, 1, self.hidden_size))

In [46]:
def char2tensor(string):
    tensor = torch.zeros(len(string)).long()
    for char in range(len(string)):
        if string[char] in all_characters:
            tensor[char] = all_characters.index(string[char])
        else:
            tensor[char] = 94 #predict space if character unknown
        
    return Variable(tensor)

# Let's see it in action.
print(char2tensor('Metis0123abczABC'))

tensor([48, 14, 29, 18, 28,  0,  1,  2,  3, 10, 11, 12, 35, 36, 37, 38])


In [47]:
def random_training_set():
    chunk = random_chunk()
    inp = char2tensor(chunk[:-1])
    target = char2tensor(chunk[1:])
    return inp, target

In [48]:
def evaluate(model, prime_str='A', predict_len=100, temperature=0.8):
    
    hidden = model.init_hidden()
    prime_input = char2tensor(prime_str)
    predicted = prime_str
    
    # use priming string to build up hidden state
    
    for p in range(len(prime_str) - 1):
        _, hidden = model(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = model(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char2tensor(predicted_char)
        
    return predicted

In [49]:
# helper function
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m, s)

In [50]:
# The actual training part
def train(inp, target):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    for char in range(chunk_len):
        output, hidden = model(inp[char], hidden)
        loss += criterion(output, target[char].unsqueeze(0))

    loss.backward()
    model_optimizer.step()

    return loss.data.item() / chunk_len

In [51]:
# # parameters
# n_epochs = 1000
# print_every = 100
# plot_every = 10
# hidden_size = 256
# n_layers = 2
# learning_rate = 0.001

# # model declaration
# model = RNN(n_characters, hidden_size, n_characters, n_layers)
# model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# start = time.time()
# all_losses = []
# loss_avg = 0

# for epoch in range(1, n_epochs + 1):
#     loss = train(*random_training_set())       
#     loss_avg += loss

#     if epoch % print_every == 0:
#         print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
#         print(evaluate(model, 'A ', 100), '\n')

#     if epoch % plot_every == 0:
#         all_losses.append(loss_avg / plot_every)
#         loss_avg = 0

In [52]:
# import matplotlib.pyplot as plt
# import matplotlib.ticker as ticker
# %matplotlib inline

# plt.figure()
# plt.plot(all_losses)

In [53]:
# with open('models/all_rappers_rnn_generator.pkl', 'wb') as picklefile:
#     pickle.dump(model, picklefile)

In [54]:
# Evaluate model from this notebook
print(evaluate(model, 'To be or not to be', predict_len=200, temperature=0.5))

To be or not to be you and the rapper i fucking and a bad the cride and i need some my hold a fuckin for me not a nigga never chass you here the come should all a man and nothing im started a new where the whole nigga 


In [55]:
# Evaluate model from this notebook
print(evaluate(model, 'I miss you girl come back ', predict_len=200, temperature=0.5))

I miss you girl come back the ome you let the real the hallers me it someal who time ballere should nigga word ball the was a livin the shit me shit fam and she new in the on the bottom boottas when the sested im the all bitch


In [56]:
# Evaluate model from this notebook
print(evaluate(model, "I'm the realest rapper alive", predict_len=200, temperature=0.5))

I'm the realest rapper alive nigga what im on on the say the money back in the best and you the real here the wild on change right and and i wanna the never she real for the for a real i dont wai new to in the bad i fake running


In [57]:
# Evaluate model from this notebook
print(evaluate(model, "Smoke a lot of weed", predict_len=200, temperature=0.4))

Smoke a lot of weed and show and its so so my niggas in the straight so she be and and im and im a new some the started the fam and i wanna see the freened the was the word a partin the seast i aint be same and somebody


In [58]:
prompts = ['I miss you girl come back ', 'To be or not to be: ', "Started from the bottom", "I'm the realest rapper alive",
           "Smoke a lot of weed", "Baby come", 'Date a scientist', 'Butter my biscuit', 'Scramble my ', 'I sleep ',
           'Green eggs and ', 'Rollin in my', 'Drop top', 'Math is fun', 'Sentences', 'Coffee in the', 'Table that',
           'Homies', 'From the streets', 'Maserati', 'Air force one', 'President', 'Trump']

all_rappers_rnn_verses = []

for i in range(len(prompts)):
    all_rappers_rnn_verses.append(evaluate(model, prompts[i], predict_len=200, temperature=0.5))
    
    
all_rappers_rnn_verses

['I miss you girl come back like the teal here are now im so with me all me to do me the rolling back of the nigga and should shot a real sall good life now the math i aint for a hold im and hell im for from the fuck the nigga i',
 'To be or not to be: in the can some what that need for the shit i said starting of the with a like a startin the right i wanna baby no im we girl a the better now now i aint and me and the started we love i on you was wh',
 'Started from the bottom a fall nigga never side nigga bitch like a see no my got thas now it nother see the bottom back and im bad a not or i am and it me a like in for the back now on shit everybody some and inters from th',
 "I'm the realest rapper alive and she and hold i way the see i got and you need you can she got for and a fuck of me shat we wanna change of she killin the started girl for a nigga hell me shit now and the like a man my talking i",
 'Smoke a lot of weed of for the wrove that fall the stop you like me car the real 

In [59]:
columns = ['generated_verses']

all_rappers_generated_df = pd.DataFrame(all_rappers_rnn_verses, columns=columns)

In [60]:
all_rappers_generated_df['artist'] = 'Combined'

all_rappers_generated_df

,generated_verses,artist
0,I miss you girl come back like the teal here a...,Combined
1,To be or not to be: in the can some what that ...,Combined
2,Started from the bottom a fall nigga never sid...,Combined
3,I'm the realest rapper alive and she and hold ...,Combined
4,Smoke a lot of weed of for the wrove that fall...,Combined
5,Baby come it though a like i dont the come tha...,Combined
6,Date a scientisten some real and of misters ma...,Combined
7,Butter my biscuit to the rap im in a money the...,Combined
8,Scramble my real from the way the weard for i ...,Combined
9,I sleep west be could the wanna fall in the sh...,Combined


In [61]:
with open('data/all_rappers_generated_raps.pkl', 'wb') as picklefile:
    pickle.dump(all_rappers_generated_df, picklefile)